##Machine Learning for finding the best location to open a Gas Station in the City of Miami, Florida

In [26]:
# importing all neccessary dependencies 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
import requests
import folium
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize #for tranforming JSON file into a pandas dataframe

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes #for four square API

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [27]:
!pip install geopy
from geopy.geocoders import Nominatim # converting an address into latitude and longitude values

In [28]:
# install either version 3.4.2 of lxml or 4.4.2. Restart the kernel after installing if wikipedia table is not extracted
!pip install lxml==4.4.2

In [29]:
## Scraping the List of Neighborhoods of Miami from Wikipedia and Preparing a Dataframe

In [30]:
# scrapping table of Miami neighborhoods from wikipedia
url  = "https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [31]:
df_Miami = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_Miami.head(5)

,Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257


In [32]:
# Dropping unncessary column from the wikipedia table
df=df_Miami.drop(['Demonym', 'Population2010','Population/Km²','Sub-neighborhoods'], axis=1)
df.head()

,Neighborhood,Coordinates
0,Allapattah,25.815-80.224
1,Arts & Entertainment District,25.799-80.190
2,Brickell,25.758-80.193
3,Buena Vista,25.813-80.192
4,Coconut Grove,25.712-80.257


In [33]:
#Dropoing the the rows on which the Coordinates is empty
df.dropna(subset=['Coordinates'], inplace=True)

In [34]:
#Identifying Neighborhood is empty but coordinates exists
blank_neighborhood = df[df['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is blank: {}'.format(blank_neighborhood))

Number of rows on which Neighborhood column is blank: 0


In [35]:
# Splitting the coordinates column into Latitude and Longitude + Dropping the coordinates column
df['Latitude']= df['Coordinates'].str[0:6]
df['Longitude']=df['Coordinates'].str[6:13]
df.drop(columns =["Coordinates"], inplace = True) 
df.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257


In [36]:
#Making sure that Latitude and Longitude columns are of numeric type
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

In [37]:
# Finding the shape of the dataset
print('The shape of the dataset is:',df.shape)

The shape of the dataset is: (24, 3)


In [38]:
## Checking whether we got the correct coordinates of Miami

In [39]:
# Getting the Latitude and Longitude of Miami city
address = 'Miami City, FL'

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Miami City are 25.7742658, -80.1936589.


In [40]:
# Creating a map of city of Miami showing neighborhoods
map_Miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# adding markers to map
for lat, lng, neighborhood in zip (df['Latitude'], 
                                  df['Longitude'], 
                                  df['Neighborhood']):
  
 folium.CircleMarker(
        [lat, lng],
        radius=5,
        fill_opacity=0.7).add_to(map_Miami)  
    
map_Miami

In [41]:
# Exploring venues of Miami using Four Square API

In [42]:
# Feeding the Square Credentials to use
CLIENT_ID = 'PMFIVV2KEJNCTW15ZH0LPDXT1OVNOX0YZYAW3LAWM2GGOV5B' # your Foursquare ID
CLIENT_SECRET = 'AZWCS4U44QG4ZQZKXK34NVZPQ1KDK5I0X012XFL4GNI5LDDC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PMFIVV2KEJNCTW15ZH0LPDXT1OVNOX0YZYAW3LAWM2GGOV5B
CLIENT_SECRET:AZWCS4U44QG4ZQZKXK34NVZPQ1KDK5I0X012XFL4GNI5LDDC


In [43]:
# Exploring the latitude and longitude of a certain neighborhood of Miami 
N_latitude = df.loc[22, 'Latitude'] # Neighborhood latitude value
N_longitude = df.loc[22, 'Longitude'] # Neighborhood longitude value

N_name = df.loc[2, 'Neighborhood'] # Neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(N_name, 
                                                               N_latitude, 
                                                               N_longitude))

Latitude and longitude values of Brickell are 25.736, -80.155.


In [44]:
# setting up result limit and defining radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    N_latitude, 
    N_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json() # make the GET request

In [45]:
# function for finding the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
# Finding the venues of the neighborhood and cleaning the Json file

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flattening JSON

# filtering columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1) # filtering the category for each row

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns] # Preparing columns

nearby_venues.head()

,name,categories,lat,lng
0,Virginia Key Beach Park,Park,25.737444,-80.162234
1,WetLab,Dive Bar,25.731700,-80.163168
2,Key Biscayne Traffic Detail,Moving Target,25.729902,-80.158250
3,SALT Waterfront Restaurant,Cafeteria,25.731698,-80.163226


In [47]:
# Number of venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [48]:
## Exploring the Neighborhoods of Miami
#Getting venues for all neighborhoods of Miami
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
Miami_venues = getNearbyVenues(names=df['Neighborhood'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude'])

Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Midtown
Overtown
Park West
The Roads
Upper Eastside
Venetian Islands
Virginia Key
West Flagler
Wynwood


In [50]:
# Size of dataframe
Miami_venues.shape

(1378, 7)

In [51]:
#Number of venues per neighborhood of Miami
Miami_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allapattah,11,11,11,11,11,11
Arts & Entertainment District,100,100,100,100,100,100
Brickell,100,100,100,100,100,100
Buena Vista,100,100,100,100,100,100
Coconut Grove,12,12,12,12,12,12
Coral Way,24,24,24,24,24,24
Design District,100,100,100,100,100,100
Downtown,100,100,100,100,100,100
Edgewater,100,100,100,100,100,100


In [52]:
#finding the list of categories
Miami_venues['Venue Category'].unique()[:100]

array(['Nightclub', 'Discount Store', 'Grocery Store', 'Video Store',
       'Park', 'Food & Drink Shop', 'Food Truck', 'Spanish Restaurant',
       'Cosmetics Shop', 'Cupcake Shop', 'Wine Shop', 'Restaurant', 'Spa',
       'Sandwich Place', 'French Restaurant', 'Gym', 'Pizza Place',
       'Gun Range', 'Ice Cream Shop', 'Paper / Office Supplies Store',
       'Beer Garden', 'Donut Shop', 'Cuban Restaurant',
       'Tapas Restaurant', 'Salon / Barbershop', 'Moving Target',
       'Seafood Restaurant', 'Brewery', 'New American Restaurant',
       'Art Gallery', 'Peruvian Restaurant', 'Gym / Fitness Center',
       'Empanada Restaurant', 'Burger Joint', 'Mexican Restaurant',
       'Breakfast Spot', 'Dessert Shop', 'Bar', 'Asian Restaurant',
       'Dog Run', 'Juice Bar', 'Smoothie Shop', 'Poke Place',
       'Indie Movie Theater', 'Coffee Shop', 'Neighborhood',
       'Cocktail Bar', 'South American Restaurant',
       'Residential Building (Apartment / Condo)', 'Public Art',
       'Bo

In [53]:
# Exploring the results containing the word "Italian Restaurants"
"Gas Station" in Miami_venues['Venue Category'].unique()

True

In [54]:
## Analzing Each Neighborhood
# one hot encoding
Miami_onehot = pd.get_dummies(Miami_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighborhood column to dataframe
Miami_onehot['Neighborhood'] = Miami_venues['Neighborhood'] 

# moving neighborhood column to the first column
fixed_columns = [Miami_onehot.columns[-1]] + list(Miami_onehot.columns[:-1])
Miami_onehot = Miami_onehot[fixed_columns]


Miami_onehot.head()

,Yoga Studio,Airport Service,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,

In [55]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Miami_grouped = Miami_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(Miami_grouped.shape)
Miami_grouped

(24, 195)


,Neighborhood,Yoga Studio,Airport Service,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Night Market,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Allapattah,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.090909,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.090909,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.00,0.00,0.090909,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.090909,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.090909,0.000000,0.00,0.181818,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0

In [56]:
len(Miami_grouped[Miami_grouped["Gas Station"] > 0])

7

In [57]:
M_grouped = Miami_grouped[["Neighborhood","Gas Station"]]
M_grouped.head()

,Neighborhood,Gas Station
0,Allapattah,0.0
1,Arts & Entertainment District,0.0
2,Brickell,0.0
3,Buena Vista,0.0
4,Coconut Grove,0.0


In [58]:
## K-means clustering

In [59]:
# set number of clusters
toclusters = 3

to_clustering = M_grouped.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2], dtype=int32)

In [60]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Cluster = M_grouped.copy()

# add clustering labels
Cluster["Cluster Labels"] = kmeans.labels_

In [61]:
Cluster.rename(columns={"Neighborhood": "Neighborhood"}, inplace=True)
Cluster.head(20)

,Neighborhood,Gas Station,Cluster Labels
0,Allapattah,0.000000,1
1,Arts & Entertainment District,0.000000,1
2,Brickell,0.000000,1
3,Buena Vista,0.000000,1
4,Coconut Grove,0.000000,1
5,Coral Way,0.000000,1
6,Design District,0.000000,1
7,Downtown,0.000000,1
8,Edgewater,0.000000,1
9,Flagami,0.033333,2


In [62]:
# merging Miami_grouped with Cluster Labels data to add latitude/longitude for each neighborhood
Cluster = Cluster.join(Miami_venues.set_index("Neighborhood"), on="Neighborhood")

print(Cluster.shape)
Cluster.head(5)

(1378, 9)


,Neighborhood,Gas Station,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allapattah,0.0,1,25.815,-80.224,Club Tipico Dominicano,25.809557,-80.218593,Nightclub
0,Allapattah,0.0,1,25.815,-80.224,Family Dollar,25.807208,-80.223503,Discount Store
0,Allapattah,0.0,1,25.815,-80.224,Winn-Dixie,25.808179,-80.224911,Grocery Store
0,Allapattah,0.0,1,25.815,-80.224,Redbox,25.808122,-80.224456,Video Store
0,Allapattah,0.0,1,25.815,-80.224,Charles Hadley Pool,25.819565,-80.216753,Park


In [63]:
# sorting the output by Cluster Labels
print(Cluster.shape)
Cluster.sort_values(["Cluster Labels"], inplace=True)
Cluster.head()

(1378, 9)


,Neighborhood,Gas Station,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
10,Grapeland Heights,0.018868,0,25.792,-80.258,The Rickenbacker,25.794437,-80.259848,Bar
12,Little Haiti,0.018519,0,25.824,-80.191,Morningside Tennis Center,25.826468,-80.181455,Tennis Court
13,Little Havana,0.012048,0,25.773,-80.215,Pinolandia,25.774787,-80.214641,Latin American Restaurant
13,Little Havana,0.012048,0,25.773,-80.215,Doce Provisions,25.767856,-80.214189,Cuban Restaurant
13,Little Havana,0.012048,0,25.773,-80.215,Viva México,25.768581,-80.214614,Mexican Restaurant


In [64]:
## Analysis of each cluster through visualizing


In [65]:
# Loading Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11,tiles='cartodbpositron')

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [67]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster['Neighborhood Latitude'], Cluster['Neighborhood Longitude'], Cluster['Neighborhood'], Cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters)
       
map_clusters

In [68]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [69]:
#Cluster 0
Cluster.loc[Cluster['Cluster Labels'] == 0]

,Neighborhood,Gas Station,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
10,Grapeland Heights,0.018868,0,25.792,-80.258,The Rickenbacker,25.794437,-80.259848,Bar
12,Little Haiti,0.018519,0,25.824,-80.191,Morningside Tennis Center,25.826468,-80.181455,Tennis Court
13,Little Havana,0.012048,0,25.773,-80.215,Pinolandia,25.774787,-80.214641,Latin American Restaurant
13,Little Havana,0.012048,0,25.773,-80.215,Doce Provisions,25.767856,-80.214189,Cuban Restaurant
13,Little Havana,0.012048,0,25.773,-80.215,Viva México,25.768581,-80.214614,Mexican Restaurant
13,Little Havana,0.012048,0,25.773,-80.215,Sedano's,25.774067,-80.215799,Grocery Store
13,Little Havana,0.012048,0,25.773,-80.215,Marlins Park,25.778278,-80.219891,Baseball Stadium
13,Little Havana,0.012048,0,25.773,-80.215,Con Sabor A Mexico,25.767875,-80.214595,Mexican Restaurant
13,Little Havana,0.012048,0,25.773,-80.215,Azucar Ice Cream Company,25.765815,-80.219711,Ice Cream Shop
13,Little Havana,0.012048,0,25.773,-80.215,El Palacio De Los Jugos,25.773810,-80.213158,Latin American Restaurant


In [70]:
#Cluster 1
Cluster.loc[Cluster['Cluster Labels'] == 1]

,Neighborhood,Gas Station,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
14,Lummus Park,0.0,1,25.777,-80.201,Riverside Park,25.770503,-80.206589,Playground
14,Lummus Park,0.0,1,25.777,-80.201,Manuel Artime Theatre,25.772528,-80.209579,Theater
14,Lummus Park,0.0,1,25.777,-80.201,Historic Miami River Hotel,25.772610,-80.201454,Hotel
14,Lummus Park,0.0,1,25.777,-80.201,Shoe Gallery Boutique,25.776937,-80.192194,Sporting Goods Shop
14,Lummus Park,0.0,1,25.777,-80.201,SUBWAY,25.774384,-80.191756,Sandwich Place
14,Lummus Park,0.0,1,25.777,-80.201,Groovin Bean Coffee Bar & Lounge,25.781890,-80.199544,Coffee Shop
14,Lummus Park,0.0,1,25.777,-80.201,MDT Metrorail - Historic Overtown/Lyric Theatr...,25.781167,-80.196303,Light Rail Station
14,Lummus Park,0.0,1,25.777,-80.201,Los Paisanos,25.773433,-80.208176,Restaurant
14,Lummus Park,0.0,1,25.777,-80.201,Black Police Precinct & Courthouse Museum,25.784253,-80.202778,History Museum
14,Lummus Park,0.0,1,25.777,-80.201,Mova Lounge Brickell,25.769698,-80.199038,Lounge


In [71]:
#Cluster 2
Cluster.loc[Cluster['Cluster Labels'] == 2]

,Neighborhood,Gas Station,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
9,Flagami,0.033333,2,25.762,-80.316,Esevagonte Eatery,25.763281,-80.306335,Italian Restaurant
22,West Flagler,0.041667,2,25.775,-80.243,Micro Teatro,25.769611,-80.239557,Theater
22,West Flagler,0.041667,2,25.775,-80.243,Macanoco & Co.,25.770411,-80.246506,Kids Store
22,West Flagler,0.041667,2,25.775,-80.243,Donut Divas & Fun Foods,25.779029,-80.250676,Cupcake Shop
22,West Flagler,0.041667,2,25.775,-80.243,CVS pharmacy,25.779116,-80.239389,Pharmacy
22,West Flagler,0.041667,2,25.775,-80.243,Subway,25.782665,-80.239031,Sandwich Place
9,Flagami,0.033333,2,25.762,-80.316,Mi Gallito Bakery,25.761785,-80.325676,Bakery
9,Flagami,0.033333,2,25.762,-80.316,Pan de Bono,25.762874,-80.317429,Bakery
9,Flagami,0.033333,2,25.762,-80.316,El Gallego Restaurant,25.763045,-80.311695,Spanish Restaurant
9,Flagami,0.033333,2,25.762,-80.316,El Rincon del Coqui,25.765737,-80.325079,Food Truck


In [ ]:
# Getting the Latitude and Longitude of Miami city
address = 'Miami City, FL'

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Miami City are 25.7742658, -80.1936589.


In [ ]:
map_Miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# adding markers to map
for lat, lng, neighborhood in zip (df['Latitude'], 
                                  df['Longitude'], 
                                  df['Neighborhood']):
  
 folium.CircleMarker(
        [lat, lng],
        radius=5,
        fill_opacity=0.7).add_to(map_Miami)  
    
map_Miami

In [ ]:
  label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)